[실습] titanic3.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요

In [73]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

# 업로드된 파일을 데이터프레임으로 읽기
file_path = list(uploaded.keys())[0]
df = pd.read_csv(file_path)

Saving titanic3.csv to titanic3 (5).csv


In [74]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [40]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [15]:
# 결측치 확인
print(df.isnull().sum())

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64


In [75]:
# 필요 없는 열 제거
# pclass 와 fare 중복이므로 pclass 삭제 => 어느 컬럼을 제거하면 좋을지 탐색해야함
df.drop(columns=['name','ticket','body','home.dest','boat','pclass'], inplace = True)
df.head()

,survived,sex,age,sibsp,parch,fare,cabin,embarked
0,1,female,29.00,0,0,211.3375,B5,S
1,1,male,0.92,1,2,151.5500,C22 C26,S
2,0,female,2.00,1,2,151.5500,C22 C26,S
3,0,male,30.00,1,2,151.5500,C22 C26,S
4,0,female,25.00,1,2,151.5500,C22 C26,S


In [76]:
# 결측치 처리
df['age'].fillna(df['age'].mean(), inplace = True)
df['fare'].fillna(df['fare'].mean(), inplace = True)
df['embarked'].fillna(df['embarked'].mode()[0], inplace = True) # 최빈값이 여러개라면 첫번째 값으로

df.drop(columns=['cabin'],inplace = True) #결측치가 많은 경우 열 제거

In [77]:
# age_group을 6개 범주로 확대
bins = [0,5,12,18,35,60,100]
labels = ['Infant','Child','Teenager','Yound Adult','Adult','Senior']
df['age_group'] = pd.cut(df['age'], bins=bins, labels = labels)
df.drop(columns=['age'], inplace = True)
df.head()

,survived,sex,sibsp,parch,fare,embarked,age_group
0,1,female,0,0,211.3375,S,Yound Adult
1,1,male,1,2,151.5500,S,Infant
2,0,female,1,2,151.5500,S,Infant
3,0,male,1,2,151.5500,S,Yound Adult
4,0,female,1,2,151.5500,S,Yound Adult


In [78]:
# sibap와 parch 합쳐서 가족 규모 변수 생성
df['family_size'] = df['sibsp']+df['parch']+1 #자기자신 포함
df.drop(columns=['sibsp','parch'],inplace = True)

In [45]:
df.head()

,survived,sex,fare,embarked,age_group,family_size
0,1,female,211.3375,S,Yound Adult,1
1,1,male,151.5500,S,Infant,4
2,0,female,151.5500,S,Infant,4
3,0,male,151.5500,S,Yound Adult,4
4,0,female,151.5500,S,Yound Adult,4


In [79]:
df.head()# fare를 3개 범주로 확대
fare_bins=[0,30,100,600]
fare_labels=['low','medium','high']
df['fare_group']=pd.cut(df['fare'], bins=fare_bins, labels=fare_labels)
df.drop(columns=['fare'], inplace=True)

In [80]:
# 범주형 변수를 더미 변수로 변환할 열 목록
categorical_columns = ['age_group','fare_group','sex','embarked']

# 반복문을 사용하여 범주형 변수를 더미 변수로 변환
for column in categorical_columns:
    # 첫 번재 카테고리는 drop_first=True로 열 제거
    df = pd.get_dummies(df, columns=[column], drop_first = True)

In [81]:
# 타겟 변수와 피처 변수 설정
X = df.drop('survived', axis=1)
y = df['survived']

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


(1047, 11) (262, 11) (1047,) (262,)
Accuracy: 0.767175572519084
Confusion Matrix:
 [[123  21]
 [ 40  78]]
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.85      0.80       144
           1       0.79      0.66      0.72       118

    accuracy                           0.77       262
   macro avg       0.77      0.76      0.76       262
weighted avg       0.77      0.77      0.76       262



Task1. train.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요.

In [52]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

# 업로드된 파일을 데이터프레임으로 읽기
file_path = list(uploaded.keys())[0]
df = pd.read_csv(file_path)

Saving train.csv to train.csv


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
